# GSA - Genetic Stability Analyzer

In order to speed up processing time when running classification algorithms, it is often useful to choose only the most "best" genes to use.  There are various algorithms available to choose genes, however here we use Chi2 Select K best.  K is how many genes you wish to use for testing stability.  More genese is usually better, however again in order to speed up processing time we limit the number of genes used.  This program allows you to set a min and max number of genes and an interval.  This will in turn setup numpy arrays with class and the select number of genes for further processing by FASTR and FASTrand.

### Libraries
Must be pre-installed.  Recommended to use virtual environment.

In [1]:
import numpy as np
from typing import Optional
from random import sample, choice, uniform
from os import path, getcwd, makedirs
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import lsqr
from math import sqrt, floor
from sklearn import svm
from multiprocessing import Pool, cpu_count
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

/Users/terek/.virtualenvs/py370/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Methods and Classes
This section defines all classes and methods used.  The corresponding methods and classes for each .py file found in `main/Python` are described here.

### NBC.py

The NBC.py file contains two classes: Model and NBC.  The Network-based supervised classification technique (NBC) is described in [Ahmet Ay et al](http://journals.sagepub.com/doi/abs/10.4137/CIN.S14025).  Briefly, for each gene, a model is constructed from the gene's neighbors.  The model is a function of the form:

gene_expressipn @ g = neigh1X + neigh2X +....neighNx + C

The set of expressions for all genes creates the expressio nmodel.

In [2]:
class Model:
    """The model constructed for a given class"""

    def __init__(self, X, label, epsilon=0.8):
        """Initializes the model for a given class.

        :param X: Training data for model. If array or matrix, shape [n_samples, n_features].
        :param label: Class label for the data.
        :param epsilon: epsilon value correlation cutoff
        """
        self._label = label
        self.X = np.array(X)

        correlations = np.corrcoef(self.X, y=None, rowvar=False)

        # Note: the mask is the graph.
        self.mask = (np.absolute(correlations) > epsilon)

        pool = Pool(processes=cpu_count())
        self.coefficients = pool.map(self.solver, [gene for gene in range(len(correlations))])
        pool.terminate()

        self.coefficients = np.array(self.coefficients)

    def solver(self, gene):
        """Uses least-square solver to compute coefficients for Ax=b, where
        A is an equation list created from the neighbors of a gene and b is
        the value of the gene.

        :param gene: The index of the gene to be solved for.
        :return: The coefficients of the equation (x) in Ax=b
        """
        mask = self.mask[gene]
        A = []
        b = []
        for sample in self.X:
            neighbors = [sample[neighbor] if (mask[neighbor] and (gene != neighbor))
                         else 0 for neighbor in range(len(mask))]
            neighbors.append(1)
            A.append(neighbors)
            b.append(sample[gene])
        A = np.array(A)
        b = np.array(b)
        x = lsqr(A, b)[0]
        return x.tolist()

    def expression(self, sample):
        """Returns a hypothetical expression level for a given sample.

        :param sample: Test sample
        :return: The hypothetical expression level of a given sample.
        """
        expression = []
        for gene in range(len(self.coefficients)):
            level = 0  # the expression level of a gene
            for neighbor in range(len(self.mask) - 1):
                level += self.coefficients[gene][neighbor] * sample[neighbor]
            level += self.coefficients[gene][len(self.mask)]
            expression.append(level)
        return np.array(expression)

    def label(self):
        """Returns the class label of the model.

        :return: The class label of the model.
        """
        return self._label


class NetworkBasedClassifier:
    """Classifier implementing the Network-based Classifier."""

    def __init__(self, epsilon=0.8):
        """Initializes the classifier.

        :param epsilon: epsilon value correlation cutoff.
        """
        self.models = []
        self.epsilon = epsilon

    def fit(self, X, y):
        """Fit the model using X as training data and y as target values.

        :param X: Training data. If array or matrix, shape [n_samples, n_features].
        :param y: Target values of shape = [n_samples]
        """
        y = np.array(y)
        X = np.array(X)
        for label in Counter(y):
            a_class = np.where(y == label)
            self.models.append(Model([X[i] for i in a_class[0]], label, self.epsilon))

    def predict(self, X):
        """Predict the class labels for the provided data.

        :param X: Test samples.

        :return: lass labels for each data sample.
        """
        pool = Pool(processes=cpu_count())
        classifications = pool.map(self.classification, [sample for sample in X])
        pool.terminate()
        return np.array(classifications)

    def score(self, X, y):
        """Returns the mean accuracy on the given test data and labels.

        :param X: Test samples.
        :param y: True labels for X.

        :return: Mean accuracy of self.predict(X) wrt. y.
        """
        y = np.array(y)
        X = np.array(X)
        correct = np.asarray(self.predict(X) == y)
        return np.sum(correct) / correct.shape[0]

    def classification(self, sample):
        """Returns the classification of the sample.

        :param sample: Test sample
        :return: The class label of the sample.
        """
        errors = []
        for model in self.models:
            error = sqrt(mean_squared_error(sample, model.expression(sample)))
            errors.append(error)
        min_index = errors.index(min(errors))
        return self.models[min_index].label()


In [3]:
class Estimator:
    """Describes the classifier chooser."""

    # Classifiers included:
    NBC = 0
    KNN = 1
    SVM = 2
    RF = 3
    NB = 4
    
    def __init__(self, _type, epsilon=0.8, k=1):
        """Initializes the classifier type.

        :param _type: The type of classifier to initialize.
        :param epsilon: Used in NBC (See nbc.py for more details).
        :param K: Used in KNeighborsClassifier (See documentation for more details).
        """
        self._type = _type

        # create the classifier
        if _type == Estimator.NBC:
            self._classifier = NetworkBasedClassifier(epsilon)
            self._name = "NBC"
        elif _type == Estimator.KNN:
            self._classifier = KNeighborsClassifier(k)
            self._name = "KNN"
        elif _type == Estimator.SVM:
            self._classifier = svm.LinearSVC()
            self._name = "SVM"
        elif _type == Estimator.RF:
            self._classifier = RandomForestClassifier()
            self._name = "RF"
        elif _type == Estimator.NB:
            self._classifier = GaussianNB()
            self._name = "NB"
    
    @property
    def type(self):
        """Returns the type of the classifier.

        :return: The type of the classifier
        """
        return self._type
    
    @property
    def name(self):
        """Returns the name of the classifier.

        :return: The string name of the classifier.
        """
        return self._name
    
    def fit(self, X, y):
        """Fit the model using X as training data and y as target values.

        :param X: Training data. If array or matrix, shape [n_samples, n_features].
        :param y: Target values of shape = [n_samples]
        """
        self._classifier.fit(X, y)
        
    def score(self, X, y):
        """Returns the mean accuracy on the given test data and labels.

        :param X: Test samples
        :param y: True labels for X
        :return: Mean accuracy of self.predict(X) wrt. y.
        """
        return self._classifier.score(X, y)
        
    def predict(self, X):
        """Predict the class labels for the provided data.

        :param X: Test samples.
        :return: lass labels for each data sample.
        """
        return self._classifier.predict(X)

### Alter.py
The Alter.py file contains all methods and helper methods used to alter expressions.  There are currently four methods to alter expressions:

1) Greedy - uses a greed strategy to select the top k genes that will produce the largest bad accuracy
2) All - alters all genes by some percent amount.
3) Subset - alters a subset of the genes selected via chi2 value
4) RandSubset - alters a subset of the genes selected randomly


In [4]:
class AlterStrategy:
    """Describes the alteration strategy used."""
    estimator: Optional[Estimator]

    # Alter strategies included:
    ALL = 0
    SUB = 1
    RAND_SUB = 2
    GREEDY = 3

    def __init__(self,
                 _type,
                 estimator: Estimator = None,
                 X=None, y=None,
                 path=None):
        self._type = _type
        self.estimator = estimator
        self.X = X
        self.y = y
        if self.type == AlterStrategy.ALL:
            self._name = "ALL"
        elif self.type == AlterStrategy.SUB:
            self._name = "SUB"
        elif self.type == AlterStrategy.RAND_SUB:
            self._name = "RANDSUB"
        elif self.type == AlterStrategy.GREEDY:
            self._name = "GREEDY"
            filename = '{}greedyRank.npy'.format(estimator.name)
            self.greedy_path = os.path.join(path, filename)
            if not os.path.exists(self.greedy_path):
                print("GREEDY RANK INITIALIZER - THIS CAN TAKE A WHILE.")
                self.estimator.fit(X, y)
                print(X.shape[1])
                to_choose = list(range(X.shape[1]))
                chosen = []
                for i in range(X.shape[1]):
                    pool = Pool(processes=cpu_count())
                    accuracies = pool.starmap(
                        self.accuracy,
                        [(X, y, chosen, idx, estimator) for idx in to_choose])
                    pool.terminate()
                    a = [x for _, x in sorted(zip(accuracies, to_choose))]
                    chosen.append(a[0])
                    to_choose.remove(a[0])
                np.save(self.greedy_path, chosen)

    @property
    def type(self):
        return self._type

    @property
    def name(self):
        return self._name

    def accuracy(self,
                 X, y,
                 chosen, idx_to_change,
                 estimator: Estimator=None):
        # TODO: run multiple times and return avg accuracy
        result = []
        for x in X:
            alt = np.copy(x)
            # alter prev chosen
            for i in chosen:
                alt[i] = 0
            # alter new gene
            alt[idx_to_change] = 0
            result.append(alt)
        result = np.array(result)
        return estimator.score(result, y)

    def alter(self, percent, X):
        """B/c genese are already ordered by chi2 rank we can choose top k to alter.

        :param percent: The percent of the subset to select from X
        :param X: Test samples.
        :return:
        """
        # high = np.amax(X)
        # low = np.amin(X)
        result = []
        k = floor(X[0].size * percent)
        if k <= 0:
            return X
        else:
            indices = []
            if self.type == AlterStrategy.RAND_SUB:
                indices = sample(range(X[0].size), k)
            elif self.type == AlterStrategy.SUB:
                indices = list(range(k))
            elif self.type == AlterStrategy.GREEDY:
                indices = np.load(self.greedy_path)[0:k:1]
            elif self.type == AlterStrategy.ALL:
                indices = list(range(X[0].size))
            for x in X:
                alt = np.copy(x)
                for i in indices:
                    if self.type == AlterStrategy.ALL:
                        offset = alt[i] * percent
                        low = alt[i] - offset
                        high = alt[i] + offset
                        alt[i] = choice([low, high])
                    else:
                        alt[i] = choice([0, alt[i] * 2])
                result.append(alt)
            return np.array(result)

    def cross_validate(self, percent=0, cv=10):
        scores = []
        skf = StratifiedKFold(cv)
        for train_index, test_index in skf.split(self.X, self.y):
            self.estimator.fit(self.X[train_index], self.y[train_index])
            accuracy = self.estimator.score(
                self.alter(percent, self.X[test_index]), self.y[test_index])
            scores.append(accuracy)
        return np.array(scores)


#### Helpers

The strategy is to choose greedily, however because of the random choice from the Accuracy method strategy, different genese may be chose, thus affecting the order in which the rank returns.  In general, the top gene should be the top gene in all cases, but as it progresses the genese can switch.

### Common.py
These are files that are helpers for the main program.  They consist of two methods:
1) A method to order the K best ranked genes.  This is used in conjunction with Atler.py Subset method.
2) A method to cross validate.  This is different than normal cross validation in that the estimator is fit to correct data while the score is derived from altered data.

In [5]:
def SelectKBestRanked(k, X, y):    
    b = SelectKBest(chi2, k).fit(X, y)
    a = b.get_support(indices = True)
    a = [x for _,x in sorted(zip(b.scores_[a],a),reverse=True)]
    return np.array(a)

### Estimatory.py

## START MAIN PROGRAM

###### Enter the series and feature_size to use
Must be all upper case. e.g. `"GSE27562"`

In [7]:
series = "GSE19804"
feature_size = 50
estimator = Estimator(Estimator.NBC)

### Get/Create Directories
Assumes this notebook is in `GenClass-Stability/main/notebooks/`

In [8]:
notebook_dir = getcwd();
main_dir = path.dirname(path.dirname(notebook_dir))
load_path = path.join(main_dir, "GSE", series)
gsa_path = path.join(main_dir,"GSA", series, str(feature_size))
if not path.exists(gsa_path):
    makedirs(gsa_path)

### Import Classes and Expressions
Load original data. Assumes SIT and custome GSE script have been run to import data.

In [9]:
classes =np.loadtxt(path.join(load_path, "classes.txt"), dtype=np.str, delimiter="\t")
exprs = np.loadtxt(path.join(load_path, "exprs.txt"), delimiter="\t")

Select K best genes for analysis.

In [10]:
a = SelectKBestRanked(feature_size, exprs, classes)

Get selected genes from expressions.

In [11]:
exprs = exprs[:, a]

Save the selected expression data for potential later use.

In [12]:
np.save(path.join(gsa_path,"exprs.npy"), exprs)
np.save(path.join(gsa_path,"classses.npy"), classes)

### Stability Test

In [13]:
alterStrat = AlterStrategy(AlterStrategy.ALL, estimator, exprs, classes, gsa_path)

In [14]:
percents = np.arange(0,1.01,0.05)
accuracies = []
stds = []
for percent in percents:
    print ("Percent:", percent)
    scores = alterStrat.cross_validate(percent, 10)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))
    accuracies.append(scores.mean())
    stds.append(scores.std())
accuracies = np.array(accuracies)
stds = np.array(stds)

result = np.column_stack((percents,accuracies,stds))

Percent: 0.0
Accuracy: 0.97 (+/- 0.04)
Percent: 0.05
Accuracy: 0.97 (+/- 0.04)
Percent: 0.1
Accuracy: 0.97 (+/- 0.04)
Percent: 0.15000000000000002
Accuracy: 0.96 (+/- 0.06)
Percent: 0.2
Accuracy: 0.95 (+/- 0.07)
Percent: 0.25
Accuracy: 0.95 (+/- 0.07)
Percent: 0.30000000000000004
Accuracy: 0.94 (+/- 0.07)
Percent: 0.35000000000000003
Accuracy: 0.89 (+/- 0.13)
Percent: 0.4
Accuracy: 0.86 (+/- 0.09)
Percent: 0.45
Accuracy: 0.87 (+/- 0.08)
Percent: 0.5
Accuracy: 0.82 (+/- 0.13)
Percent: 0.55
Accuracy: 0.83 (+/- 0.11)
Percent: 0.6000000000000001
Accuracy: 0.72 (+/- 0.12)
Percent: 0.65
Accuracy: 0.68 (+/- 0.10)
Percent: 0.7000000000000001


Process ForkPoolWorker-1761:
Process ForkPoolWorker-1763:
Process ForkPoolWorker-1762:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._targ

KeyboardInterrupt: 

In [ ]:
filename = '{}_{}_result.npy'.format(estimator.name, alterStrat.name)
print(filename)
np.save(path.join(gsa_path, filename), result)

In [ ]:
print(result)

In [ ]:
#list(zip(*list(result)))